In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f

In [0]:
df_hosa = spark.read.parquet("/mnt/bronze/hosa/departments")

df_hosb = spark.read.parquet("/mnt/bronze/hosb/departments")

In [0]:
display(df_hosa)

In [0]:
df_merged = df_hosa.unionByName(df_hosb)
display(df_merged)

In [0]:
df_merged = df_merged.withColumn("SRC_Dept_id", col('deptid')) \
    .withColumn("Dept_id", concat(col('deptid'),lit('-'),col('datasource'))) \
    .drop('deptid')

df_merged.createOrReplaceTempView('departments')


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS hc_premiumdb.silver;

create table if not exists silver.departments(
  Dept_Id string,
  SRC_Dept_Id string,
  Name string,
  datasource string,
  is_quarantined boolean
)
USING DELTA

In [0]:
%sql
truncate table hc_premiumdb.silver.departments

In [0]:
%sql
insert into silver.departments
select 
Dept_Id,
SRC_Dept_Id,
Name,
Datasource,
CASE 
  WHEN SRC_Dept_id is null or Name is null then true
  ELSE false
  END AS is_quarantined
from departments 

In [0]:
%sql
select * from silver.departments where is_quarantined=True